# day 3-2

このノートブックの実行例は[こちら(HTML版)](https://github.com/haradatm/lecture-gssm2025/blob/main/notebooks-samples/day-3-2.html)で確認できます

---

## 0. 準備

### 0.1 必要なパッケージのインストール

In [ ]:
%%capture
!sudo apt-get update
!sudo apt-get install -y automake autoconf perl libtool graphviz libgraphviz-dev
!pip install -U japanize_matplotlib pyvis pygraphviz mca

教材のダウンロード

In [ ]:
!git clone https://github.com/haradatm/lecture-gssm2025

In [ ]:
!ln -s lecture-gssm2025/notebooks/gssm_utils.py .

### 0.2 MeCab インストール (時間目安: 約3分)

In [ ]:
%%time
!bash lecture-gssm2025/scripts/install_mecab.sh >> install_mecab.log 2>&1
!tail -n 1 install_mecab.log

### 0.3 CaboCha インストール (時間目安: 約4分)

In [ ]:
%%time
!bash lecture-gssm2025/scripts/install_cabocha.sh >> install_cabocha.log 2>&1
!tail -n 1 install_cabocha.log

### 0.4 セッションの再起動

In [ ]:
import os
os.kill(os.getpid(), 9)

セッションの再起動後は,以下のセルから実行してください. **注意: これより前のセルを再度実行する必要はありません!**

---

### 0.5 辞書登録

追加したい形態素の情報を CSV ファイル(user_dic.csv)に追記する

In [ ]:
!echo '"泉質",-1,-1,1,名詞,一般,*,*,*,*,泉質,センシツ,センシツ,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!echo '"海鮮丼",-1,-1,1,名詞,一般,*,*,*,*,海鮮丼,カイセンドン,カイセンドン,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!echo '"スカイツリー",-1,-1,1,名詞,一般,*,*,*,*,スカイツリー,スカイツリー,スカイツリー,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!echo '"バスタオル",-1,-1,1,名詞,一般,*,*,*,*,バスタオル,バスタオル,バスタオル,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!cat ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv

CSVファイル(user_dic.csv)をコンパイルして辞書(user.dic)を作成する

In [ ]:
!./tools/usr/local/libexec/mecab/mecab-dict-index \
-d ./tools/usr/local/lib/mecab/dic/ipadic \
-u ./tools/usr/local/lib/mecab/dic/ipadic/user.dic \
-f utf-8 -t utf-8 \
./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv

### 0.6 確認

In [ ]:
import MeCab
tagger = MeCab.Tagger("-r ./tools/usr/local/etc/mecabrc")

print(tagger.parse("この泉質は極上です"))
print(tagger.parse("この海鮮丼は美味しいです"))
print(tagger.parse("近くにスカイツリーがあります"))
print(tagger.parse("浴室にバスタオルがありません"))

In [ ]:
import CaboCha

cp = CaboCha.Parser("-r ./tools/usr/local/etc/cabocharc")
print(cp.parse("この泉質は極上です").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("この海鮮丼は美味しいです").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("近くにスカイツリーがあります").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("浴室にバスタオルがありません").toString(CaboCha.FORMAT_LATTICE))

---

## 1. テキスト解析 (2)
### KHCoder のテキスト解析&分析

### 1.0 事前準備 (定義済み関数の読み込み)

以下のセルを**修正せず**に実行してください

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random
import numpy as np

# 再現性のために乱数を固定する
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)

# 定義済み関数をインポートする
import warnings
warnings.simplefilter('ignore')

import gssm_utils

### 1.1 データのダウンロード

以下のデータがダウンロード済みです

| ファイル名 | 件数 | データセット | 備考 |
| --- | --- | --- | --- |
| rakuten-1000-2024-2025.xlsx.zip | 10,000 | •レジャー+ビジネスの 10エリア<br>•エリアごと 1,000件 (ランダムサンプリング)<br>•期間: 2024/1~2025 GW明け | 本講義の全体を通して使用する |

In [ ]:
# rakuten-1000-2024-2025.xlsx.zip をダウンロードする
FILE_ID = "1yKabZ7qJMRrIrP4Vtq-RrSZAqFsZriQS"
!gdown {FILE_ID}
!unzip -o rakuten-1000-2024-2025.xlsx.zip

### 1.2 データの読み込み (DataFrame型)

In [ ]:
import numpy as np
import pandas as pd

all_df = pd.read_excel("rakuten-1000-2024-2025.xlsx")
print(all_df.shape)
display(all_df.head())

### 1.3 「文書-抽出語」表 を作成する

コメント列から単語を抽出する (単語を品詞「名詞」「形容詞」「未知語」で絞り込む)

In [ ]:
# 必要ライブラリのインポート
from collections import defaultdict
import MeCab

# mecab の初期化
tagger = MeCab.Tagger("-r ./tools/usr/local/etc/mecabrc --unk-feature 未知語")

# 単語頻度辞書の初期化
word_counts = defaultdict(lambda: 0)

# 抽出語情報リストの初期化
words = []

# 半角->全角変換マクロを定義する
ZEN = "".join(chr(0xff01 + i) for i in range(94))
HAN = "".join(chr(0x21 + i) for i in range(94))
HAN2ZEN = str.maketrans(HAN, ZEN)

# ストップワードを定義する
stopwords =  ["する", "それ", "なる", "ない", "そこ", "これ" ,"ある"]
stopwords += ["湯畑"]

# データ1行ごとのループ
for index, row in all_df.iterrows():

    # 半角->全角変換した後で, mecab で解析する
    node = tagger.parseToNode(row["コメント"].translate(HAN2ZEN))

    # 形態素ごとのループ
    while node:
        # 解析結果を要素ごとにバラす
        features = node.feature.split(',')

        # 品詞1 を取り出す
        pos1 = features[0]

        # 品詞2 を取り出す
        pos2 = features[1] if len(features) > 1 else ""

        # 原形 を取り出す
        base = features[6] if len(features) > 6 else None

        # 原型がストップワードに含まれない単語のみ抽出する
        if base not in stopwords:

            # 「名詞-一般」
            if (pos1 == "名詞" and pos2 == "一般"):
                base = base if base is not None else node.surface
                postag = "名詞"
                key = (base, postag)

                # 単語頻度辞書をカウントアップする
                word_counts[key] += 1

                # 抽出語情報をリストに追加する
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            # 「形容動詞」
            elif (pos1 == "名詞" and pos2 == "形容動詞語幹"):
                base = base if base is not None else node.surface
                base = f"{base}だ"
                postag = "形容動詞"
                key = (base, postag)

                # 単語頻度辞書をカウントアップする
                word_counts[key] += 1

                # 抽出語情報をリストに追加する
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            # 「形容詞」
            elif pos1 == "形容詞":
                base = base if base is not None else node.surface
                postag = "形容詞"
                key = (base, postag)

                # 単語頻度辞書をカウントアップする
                word_counts[key] += 1

                # 抽出語情報をリストに追加する
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            # 「未知語」
            elif pos1 == "未知語":
                base = base if base is not None else node.surface
                postag = "未知語"
                key = (base, postag)

                # 単語頻度辞書をカウントアップする
                word_counts[key] += 1

                # 抽出語情報をリストに追加する
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

        # 次の形態素へ
        node = node.next

# DataFrme 型に整える
columns = [
    "文書ID",
    # "単語ID",
    "表層",
    "品詞",
    "カテゴリー",
    "エリア",
    "dict_key",
]
docs_df = pd.DataFrame(words, columns=columns)

# DataFrame を表示する
print(docs_df.shape)
display(docs_df.head())

抽出語の出現頻度をカウントする

In [ ]:
# 「文書-抽出語」 表から単語の出現回数をカウントする
word_list = []
for i, (k, v) in enumerate(sorted(word_counts.items(), key=lambda x:x[1], reverse=True)):
    word_list.append((i, k[0], v, k))

# DataFrame 型に整える
columns = [
    "単語ID",
    "表層",
    "出現頻度",
    "dict_key"
]

# DataFrame を表示する
word_counts_df = pd.DataFrame(word_list, columns=columns)
print(word_counts_df.shape)
display(word_counts_df.head(10))

単語IDを紐つける (出現回数 Top 150語のみ抽出する)

In [ ]:
# 「単語出現回数」 表から出現回数Top 150語のみ抽出する
word_counts_150_df = word_counts_df[0:150]

# 「文書-抽出語」 表も出現回数Top 150語のみに絞り込む
merged_df = pd.merge(docs_df, word_counts_150_df, how="inner", on="dict_key", suffixes=["", "_right"])
docs_150_df = merged_df[["文書ID", "単語ID", "表層", "品詞", "カテゴリー", "エリア", "dict_key"]]

# DataFrame を表示する
# print(docs_150_df.shape)
display(docs_150_df)

「文書-抽出語」表 を作成する

In [ ]:
# 「単語出現回数」 表から出現回数Top 75語のみ抽出する
word_counts_75_df = word_counts_df[0:75]

# 「文書-抽出語」 表も出現回数Top 75語のみに絞り込む
merged_df = pd.merge(docs_df, word_counts_75_df, how="inner", on="dict_key", suffixes=["", "_right"])
docs_75_df = merged_df[["文書ID", "単語ID", "表層", "品詞", "カテゴリー", "エリア", "dict_key"]]

# 「カテゴリー,エリア」でクロス集計する
cross_75_df = pd.crosstab(
    [
        docs_75_df['カテゴリー'],
        docs_75_df['エリア'],
        docs_75_df['文書ID']
    ],
    docs_75_df['単語ID'], margins=False
)
cross_75_df.columns = word_counts_75_df["表層"]

# DataFrame を表示する
print(cross_75_df.shape)
display(cross_75_df)

「文書-抽出語」 表を {0,1} に変換する

In [ ]:
# 「文書-抽出語」 表を {0,1} に変換する
cross_75_df[cross_75_df > 0] = 1

# DataFrame を表示する
print(cross_75_df.shape)
display(cross_75_df)

In [ ]:
# 「文書-抽出語」 表をファイルに出力する
cross_75_df.to_csv("cross_75.tsv", sep="\t", encoding="utf-8")

### 1.4 共起ネットワーク

共起度行列を作成する (抽出語-抽出語)

In [ ]:
# 必要ライブラリのインポート
from scipy.sparse import csc_matrix

# 共起行列を作成する
X = cross_75_df.values
X = csc_matrix(X)
Xc = (X.T * X)
# 対角成分のみにする
Xc = np.triu(Xc.toarray())

# DataFrame 型に整える
cooccur_75_df = pd.DataFrame(Xc, columns=cross_75_df.columns, index=cross_75_df.columns)

# DataFrame を表示する
print(cooccur_75_df.shape)
display(cooccur_75_df.head())

In [ ]:
# 「抽出後-抽出語」 表をファイルに出力する
cooccur_75_df.to_csv("cooccur_75.tsv", sep="\t", encoding="utf-8", index=True)

#### (a) 共起ネットワーク (共起度の高いエッジを残す)

In [ ]:
# 抽出語の出現回数(ノードの大きさ)を取得する
word_counts = cross_75_df.sum(axis=0).values

# 共起行列(共起度)で共起ネットワーク図を作成する
gssm_utils.plot_cooccur_network(cooccur_75_df, word_counts, cooccur_75_df.values.max() * 0.05)

#### (b) 共起ネットワーク (Jaccard 係数が上位のエッジを残す)

In [ ]:
# 共起行列の中身を Jaccard 係数に入れ替える
jaccard_75_df = gssm_utils.jaccard_coef(cooccur_75_df, cross_75_df)

# 抽出語の出現回数(ノードの大きさ)を取得する
word_counts = cross_75_df.sum(axis=0).values

# 共起行列(Jaccard係数)で共起ネットワーク図を作成する
gssm_utils.plot_cooccur_network(jaccard_75_df, word_counts, np.sort(jaccard_75_df.values.reshape(-1))[::-1][60], pyvis=True)

### 1.5 対応分析

「文書-抽出語」 表を確認する

In [ ]:
# DataFrame を表示する
print(cross_75_df.shape)
display(cross_75_df.head())

「外部変数-抽出語」 クロス集計表を作成する

In [ ]:
# 「カテゴリー」のクロス集計と「エリア」のクロス集計を連結する
aggregate_75_df = pd.concat(
    [
        cross_75_df.groupby(level='カテゴリー').sum(),
        cross_75_df.groupby(level='エリア').sum()
    ]
)

# DataFrame を表示する
print(aggregate_75_df.shape)
display(aggregate_75_df)

In [ ]:
# 「外部変数-抽出語」 表をファイルに出力する
aggregate_75_df.to_csv("aggregate_75.tsv", sep="\t", encoding="utf-8", index=True)

#### (a) 対応分析プロット (ライブラリ mca を使用)

In [ ]:
# 必要ライブラリのインポート
import mca

# ライブラリ mca による対応分析
ncols = aggregate_75_df.shape[1]
mca_ben = mca.MCA(aggregate_75_df, ncols=ncols, benzecri=False)

# 行方向および列方向の値を取り出す
row_coord = mca_ben.fs_r(N=2)
col_coord = mca_ben.fs_c(N=2)

# 固有値を求める
eigenvalues = mca_ben.L
total = np.sum(eigenvalues)
# 寄与率を求める
explained_inertia = 100 * eigenvalues / total

# 行方向および列方向のラベルを取得する
row_labels = aggregate_75_df.index
col_labels = aggregate_75_df.columns

# プロットする
gssm_utils.plot_coresp(row_coord, col_coord,row_labels, col_labels, explained_inertia)

#### (b) 対応分析プロット (カイ2乗値を手計算してプロットする)

In [ ]:
table_N = aggregate_75_df.values
row_sum = table_N.sum(axis=1)
col_sum = table_N.sum(axis=0)
n = aggregate_75_df.values.sum()

# カイ2乗値を求める
expected = np.outer(row_sum, col_sum) / n
chisq = np.square(table_N - expected) / expected
residuals = (table_N - expected) / np.sqrt(expected)

# Standardized residuals
residuals = residuals / np.sqrt(n)

# Number of dimensions
nb_axes = min(residuals.shape[0]-1, residuals.shape[1]-1)

# Singular value decomposition
U, s, V = np.linalg.svd(residuals, full_matrices=True)
sv = s[:nb_axes]
u = U[:, :nb_axes]
v = V[:nb_axes, :]

# row mass
row_mass = row_sum / n

# row coord = u * sv /sqrt(row.mass)
row_coord = (u * sv) / np.sqrt(row_mass)[:, np.newaxis]

# col mass
col_mass = col_sum / n

# row coord = sv * vT /sqrt(col.mass)
col_coord = (sv * v.T) / np.sqrt(col_mass)[:, np.newaxis]

# 固有値を求める
eige_nvalue = s ** 2

# 寄与率を求める
explained_inertia = 100 * eige_nvalue / sum(eige_nvalue)

# 行方向および列方向のラベルを取得する
row_labels = aggregate_75_df.index
col_labels = aggregate_75_df.columns

# プロットする
gssm_utils.plot_coresp(row_coord, col_coord,row_labels, col_labels, explained_inertia)

#### (c) 対応分析プロット (共起頻度のままプロットする)

In [ ]:
import numpy as np

table_N = aggregate_75_df.values
table_P = table_N / aggregate_75_df.values.max()

# Singular value decomposition
U, s, V = np.linalg.svd(table_P, full_matrices=True)
sv = s[:nb_axes]
u = U[:, :nb_axes]
v = V[:nb_axes, :]

# row mass
row_mass = row_sum / n

# row coord = u * sv /sqrt(row.mass)
row_coord = (u * sv) / np.sqrt(row_mass)[:, np.newaxis]

# col mass
col_mass = col_sum / n

# row coord = sv * vT /sqrt(col.mass)
col_coord = (sv * v.T) / np.sqrt(col_mass)[:, np.newaxis]

# 固有値を求める
eige_nvalue = s ** 2

# 寄与率を求める
explained_inertia = 100 * eige_nvalue / sum(eige_nvalue)

# 行方向および列方向のラベルを取得する
row_labels = aggregate_75_df.index
col_labels = aggregate_75_df.columns

# プロットする
gssm_utils.plot_coresp(row_coord, col_coord,row_labels, col_labels, explained_inertia)

#### (d) PCAプロット (共起頻度のままプロットする)

In [ ]:
# 必要ライブラリのインポート
from sklearn.decomposition import PCA

table_N = aggregate_75_df.values

# ライブラリ PCA による主成分分析
pca = PCA()

reduced = pca.fit_transform(table_N.T)
coeff = np.transpose(pca.components_)
var_ratio = pca.explained_variance_ratio_

scalex = 1.0 / (reduced[:,0].max() - reduced[:,0].min())
scaley = 1.0 / (reduced[:,1].max() - reduced[:,1].min())
reduced[:,0] *= scalex
reduced[:,1] *= scaley

# プロットする
gssm_utils.plot_pca(coeff, reduced, row_labels, col_labels, var_ratio)

#### (e) PCAプロット (カイ2乗値をプロットする)

In [ ]:
# 必要ライブラリのインポート
from sklearn.decomposition import PCA

import numpy as np

table_N = aggregate_75_df.values
row_sum = table_N.sum(axis=1)
col_sum = table_N.sum(axis=0)
n = aggregate_75_df.values.sum()

# カイ2乗値を求める
expected = np.outer(row_sum, col_sum) / n
chisq = np.square(table_N - expected) / expected
residuals = (table_N - expected) / np.sqrt(expected)

# Standardized residuals
residuals = residuals / np.sqrt(n)

# ライブラリ PCA による主成分分析
pca = PCA()

reduced = pca.fit_transform(residuals.T)
coeff = np.transpose(pca.components_)
var_ratio = pca.explained_variance_ratio_

scalex = 1.0 / (reduced[:,0].max() - reduced[:,0].min())
scaley = 1.0 / (reduced[:,1].max() - reduced[:,1].min())
reduced[:,0] *= scalex
reduced[:,1] *= scaley

# プロットする
gssm_utils.plot_pca(coeff, reduced, row_labels, col_labels, var_ratio)

---

## 【演習】 2022~2023 データセット

### 2.0 【演習】 事前準備 (定義済み関数の読み込み)

以下のセルを**修正せず**に実行してください

In [ ]:
import warnings
warnings.simplefilter('ignore')

import gssm_utils

### 2.1 【演習】 データのダウンロード

以下のデータがダウンロード済みです

| ファイル名 | 件数 | データセット | 備考 |
| --- | --- | --- | --- |
| rakuten-1000-2022-2023.xlsx.zip | 10,000 | •レジャー+ビジネスの 10エリア<br>•エリアごと 1,000件 (ランダムサンプリング)<br>•期間: 2022/1~2023/12 | 演習用 (年度で比較する場合など) |

In [ ]:
# rakuten-1000-2022-2023.xlsx.zip をダウンロードする
FILE_ID = "13aWgWvDKpz8lGHEDQ7Z7Svitr3TF3Cs9"
!gdown {FILE_ID}
!unzip -o rakuten-1000-2022-2023.xlsx.zip

### 2.2 【演習】 データの読み込み (DataFrame型)

In [ ]:
import numpy as np
import pandas as pd

all_df = pd.read_excel("rakuten-1000-2022-2023.xlsx")
print(all_df.shape)
display(all_df.head())

### 2.3 【演習】 「文書-抽出語」表 を作成する

コメント列から単語を抽出する (単語を品詞「名詞」「形容詞」「未知語」で絞り込む)

In [ ]:
# ToDo: 1.3 の該当コードをコピーして貼り付けて実行してください

抽出語の出現頻度をカウントする

In [ ]:
# ToDo: 1.3 の該当コードをコピーして貼り付けて実行してください

単語IDを紐つける (出現回数 Top 150語のみ抽出する)

In [ ]:
# ToDo: 1.3 の該当コードをコピーして貼り付けて実行してください

「文書-抽出語」表 を作成する

In [ ]:
# ToDo: 1.3 の該当コードをコピーして貼り付けて実行してください

「文書-抽出語」 表を {0,1} に変換する

In [ ]:
# ToDo: 1.3 の該当コードをコピーして貼り付けて実行してください

### 2.4 【演習】 共起ネットワーク

共起度行列を作成する (抽出語-抽出語)

In [ ]:
# ToDo: 1.4 の該当コードをコピーして貼り付けて実行してください

#### (b) 共起ネットワーク (Jaccard 係数が上位のエッジを残す)

In [ ]:
# ToDo: 1.4 (b) の該当コードをコピーして貼り付けて実行してください

### 2.5 【演習】 対応分析

「文書-抽出語」 表を確認する

In [ ]:
# ToDo: 1.5 の該当コードをコピーして貼り付けて実行してください

「外部変数-抽出語」 クロス集計表を作成する

In [ ]:
# ToDo: 1.5 の該当コードをコピーして貼り付けて実行してください

#### (a) 対応分析プロット (ライブラリ mca を使用)

In [ ]:
# ToDo: 1.5 (a)のコードをコピーして貼り付けて実行してください